# instalation

In [91]:
!pip install hmeasure

In [104]:
# data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import Counter
from sklearn.preprocessing import StandardScaler

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import ADASYN, SMOTE, SVMSMOTE, BorderlineSMOTE
from hmeasure import h_score

# data

In [ ]:
# optional
from google.colab import drive
drive.mount('/content/drive')

In [93]:
# dataset locations
data_path = "drive/MyDrive/datasets/creditcard.csv"
data_path_2 = "drive/MyDrive/datasets/diabetes.csv"
data_path_3 = "drive/MyDrive/datasets/lung_cancer.csv"

In [195]:
def load_dataset(path):
    # load the dataset as a numpy array
    dataframe = pd.read_csv(path, na_values='?')
    # drop unnecessary columns
    #dataframe.drop(['Time'], axis=1)
    # change column names to numbers
    dataframe.columns = range(dataframe.shape[1])
    # drop rows with missing data
    dataframe = dataframe.dropna()
    # split into inputs and outputs
        
    # encode M,F in lung dataset
    if "cancer" in path:
        cleanup_nums = {0:     {"M": 0, "F": 1}}
        dataframe = dataframe.replace(cleanup_nums)

    last_id = len(dataframe.columns) - 1 # label column id
    X, y = dataframe.drop(last_id, axis=1), dataframe[last_id]
    # select categorical and numerical features
    #cat_ids = X.select_dtypes(include=['object', 'bool', 'string']).columns
    #num_ids = X.select_dtypes(include=['int64', 'float64']).columns

    # label encode the target variable to have the classes 0 and 1
    y = LabelEncoder().fit_transform(y)
    # scale data
    #X = StandardScaler().fit_transform(X.values)
    return X.values, y, dataframe # cat_ids, num_ids

# eval


In [196]:
def evaluate(X, y):
    # define models
    models = []
    models.append(('CART', DecisionTreeClassifier()))
    #models.append(('SVM', SVC(gamma='scale')))
    #models.append(('RF', RandomForestClassifier()))

    # define oversampling techniques
    oversampling = []
    oversampling.append(('NONE', None))
    oversampling.append(('SMOTE', SMOTE()))             

    for model in models:
        for over in oversampling:

            #define pipeline
            steps = [('over', over[1]), ('model', model[1])]
            pipeline = Pipeline(steps=steps)

            # 5 times 2-fold cross-validation
            cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1) # n_repeats=5

            # roc-auc
            #scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, verbose=2)#, n_jobs=-1)
            #score = mean(scores)

            # h-measure
            predictions = cross_val_predict(pipeline, X, y, cv=cv, verbose=1)#, n_jobs=-1)
            n1, n0 = y.sum(), y.shape[0]-y.sum()
            score = h_score(y, predictions, severity_ratio=(n1/n0))
            
            print(f"model: {model[0]}, type: {over[0]}, score: {score}")

In [198]:
# load chosen dataset
X, y, df = load_dataset(data_path)
print(X.shape, y.shape, Counter(y))
print(X[10])

(284807, 30) (284807,) Counter({0: 284315, 1: 492})
[ 1.00000000e+01  1.44904378e+00 -1.17633883e+00  9.13859833e-01
 -1.37566665e+00 -1.97138317e+00 -6.29152139e-01 -1.42323560e+00
  4.84558879e-02 -1.72040839e+00  1.62665906e+00  1.19964395e+00
 -6.71439778e-01 -5.13947153e-01 -9.50450454e-02  2.30930409e-01
  3.19674668e-02  2.53414716e-01  8.54343814e-01 -2.21365414e-01
 -3.87226474e-01 -9.30189652e-03  3.13894411e-01  2.77401580e-02
  5.00512287e-01  2.51367359e-01 -1.29477954e-01  4.28498709e-02
  1.62532619e-02  7.80000000e+00]


In [185]:
# optionally scale
X = StandardScaler().fit_transform(X)

In [199]:
evaluate(X,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


model: CART, type: NONE, score: 0.7202818177733689
model: CART, type: SMOTE, score: 0.7351299760899928


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   55.6s finished
